# Implementation of N-grams

**n-gram:** is a contiguous sequence of n items (which can be characters, words, or other units) that are extracted from a larger piece of text, such as a sentence or a document.

Eg: `"The quick brown fox jumps over the lazy dog."` 

Here are some examples of n-grams with different values of n:

**Unigrams (n = 1):** The, quick, brown, fox, jumps, over, the, lazy, dog<br><br>
**Bigrams (n = 2):** The quick, quick brown, brown fox, fox jumps, jumps over, over the, the lazy, lazy dog<br><br>
**Trigrams (n = 3):** The quick brown, quick brown fox, brown fox jumps, fox jumps over, jumps over the, over the lazy, the lazy dog<br><br>

N-grams are often used in natural language processing tasks such as language modeling, text classification, and machine translation. By considering the frequencies of different n-grams in a piece of text, machine learning algorithms can learn patterns and make predictions about the text. N-grams can also be used to generate new text or to detect plagiarism, among other applications.

### Let us work with the Reuters dataset

In [1]:
import nltk
import pandas as pd

# This will help view all the text
pd.set_option('display.max_colwidth', None)

NLTK provides several datasets for text classification tasks, here are some examples:

1. **Reuters Corpus:** This dataset contains 10,788 news articles from the Reuters news agency, labeled with 90 categories. The dataset is often used for topic classification and information retrieval tasks.<br><br>
2. **20 Newsgroups:** This dataset contains around 20,000 newsgroup posts from 20 different newsgroups, labeled with the newsgroup they belong to. The dataset is often used for text classification tasks, such as topic classification and sentiment analysis.<br><br>
3. **IMDB Reviews:** This dataset contains 50,000 movie reviews from the Internet Movie Database (IMDB), labeled as positive or negative. The dataset is often used for sentiment analysis and text classification tasks.<br><br>
4. **Brown Corpus:** This dataset contains 500 samples of English text from various genres, labeled with categories such as news, fiction, and reviews. The dataset is often used for genre classification and other text classification tasks.<br><br>
5. **Webtext Corpus:** This dataset contains a collection of text extracted from various websites, labeled with categories such as news, blogs, and reviews. The dataset is often used for text classification and language modeling tasks.<br><br>

In [2]:
# Load the Reuters Corpus
nltk.download('reuters')
corpus = nltk.corpus.reuters

[nltk_data] Downloading package reuters to /Users/swapnil/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [3]:
# Get the categories and fileids
categories = corpus.categories()
fileids = corpus.fileids()

In [7]:
print(f'The sample corpus has {len(categories)} unique categories and a total of {len(fileids)} articles.')

The sample corpus has 90 unique categories and a total of 10788 articles.


## Let's put this data in a dataframe

In [4]:
# Create a list of dictionaries, where each dictionary represents a document
documents = []
for fileid in fileids:
    category = corpus.categories(fileid)[0]
    raw_text = corpus.raw(fileid)
    documents.append({'fileid': fileid, 'category': category, 'text': raw_text})


# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(documents)

In [21]:
df.head()

fileid  category  \
0  test/14826     trade   
1  test/14828     grain   
2  test/14829     crude   
3  test/14832      corn   
4  test/14833  palm-oil   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [5]:
# let's count the number of words in some articles and chose the one with less words for practice
# This code will split the text in the first row by whitespace using the split() method, and then count the number of resulting words using the len() function.
# without the .split() it will count number of characters.

num_words = len(df.loc[1, 'text'].split())
print(num_words)

108


second row has 108 words, we will just use that for now

In [6]:
# use multilist so that you have a dataframe else you will have a series when it is a single row
type(df.loc[[1]])

pandas.core.frame.DataFrame

In [7]:
df= df.loc[[1]]
df.head()

,fileid,category,text
1,test/14828,grain,"CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS\n A survey of 19 provinces and seven cities\n showed vermin consume between seven and 12 pct of China's grain\n stocks, the China Daily said.\n It also said that each year 1.575 mln tonnes, or 25 pct, of\n China's fruit output are left to rot, and 2.1 mln tonnes, or up\n to 30 pct, of its vegetables. The paper blamed the waste on\n inadequate storage and bad preservation methods.\n It said the government had launched a national programme to\n reduce waste, calling for improved technology in storage and\n preservation, and greater production of additives. The paper\n gave no further details.\n \n\n"


In [8]:
df['text']

1    CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS\n  A survey of 19 provinces and seven cities\n  showed vermin consume between seven and 12 pct of China's grain\n  stocks, the China Daily said.\n      It also said that each year 1.575 mln tonnes, or 25 pct, of\n  China's fruit output are left to rot, and 2.1 mln tonnes, or up\n  to 30 pct, of its vegetables. The paper blamed the waste on\n  inadequate storage and bad preservation methods.\n      It said the government had launched a national programme to\n  reduce waste, calling for improved technology in storage and\n  preservation, and greater production of additives. The paper\n  gave no further details.\n  \n\n
Name: text, dtype: object

In [9]:
# Perform text preprocessing on the words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/swapnil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/swapnil/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/swapnil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# create a function to apply pre-processing

def preprocess_text(text):
    
    # Convert the text to lowercase
    text = text.lower()
    
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Remove stop words and punctuation
    #words = [word for word in words if word.isalpha() and word not in stop_words]
    
    import string

    # Remove punctuations from the tokens
    words = [token for token in words if token not in string.punctuation]


    # Remove stop words from the tokens
    words = [token for token in words if token not in stop_words]


    # Lemmatize the words
    words = [wordnet_lemmatizer.lemmatize(word) for word in words]
    
    # Handle ackronyms
    expanded_terms = {
    'pct': 'percentage',
    'mln':'million'
    }

    words = [expanded_terms.get(word, word) for word in words]
    
    # Convert the list of words back into a string
    preprocessed_text = ' '.join(words)
    
    return preprocessed_text

In [11]:
# apply function on text column
df['clean_text'] = df['text'].apply(preprocess_text)

In [13]:
df.clean_text

1    china daily say vermin eat 7-12 percentage grain stock survey 19 province seven city showed vermin consume seven 12 percentage china 's grain stock china daily said also said year 1.575 million tonne 25 percentage china 's fruit output left rot 2.1 million tonne 30 percentage vegetable paper blamed waste inadequate storage bad preservation method said government launched national programme reduce waste calling improved technology storage preservation greater production additive paper gave detail
Name: clean_text, dtype: object

# We can avoid this, since it is coming from reuters articles
#!pip install autocorrect
import autocorrect

spell = autocorrect.Speller(lang='en')

#tokens = ['This', 'is', 'speling', 'misstake']
corrected_tokens = [spell.autocorrect_word(token) for token in expanded_tokens]print('Corrected tokens:', corrected_tokens)

# N-Grams

In [14]:
# import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# pass Ngrams as a tuple (2,2) for bigram, (1,2) for a cobination of both unigram and bigram
cv = CountVectorizer(ngram_range=(2,2))

In [19]:
# Fit and transform the text
ngrams = cv.fit_transform(df['clean_text'])

In [16]:
# the vocabulary is stored in 'cv.vocabulary_'
# sort the keys
vocab = sorted(cv.vocabulary_.keys())

print(vocab)

['12 percentage', '19 province', '25 percentage', '30 percentage', '575 million', 'additive paper', 'also said', 'bad preservation', 'blamed waste', 'calling improved', 'china daily', 'china fruit', 'china grain', 'city showed', 'consume seven', 'daily said', 'daily say', 'eat 12', 'fruit output', 'gave detail', 'government launched', 'grain stock', 'greater production', 'improved technology', 'inadequate storage', 'launched national', 'left rot', 'method said', 'million tonne', 'national programme', 'output left', 'paper blamed', 'paper gave', 'percentage china', 'percentage grain', 'percentage vegetable', 'preservation greater', 'preservation method', 'production additive', 'programme reduce', 'province seven', 'reduce waste', 'rot million', 'said also', 'said government', 'said year', 'say vermin', 'seven 12', 'seven city', 'showed vermin', 'stock china', 'stock survey', 'storage bad', 'storage preservation', 'survey 19', 'technology storage', 'tonne 25', 'tonne 30', 'vegetable pape

#### index starts with 0. '12' has an index 0, '19' has an index 1, and so on.

In [22]:
# to print index and values iterate over the sorted keys and print the key-value pairs
for key in vocab:
    print(cv.vocabulary_[key], ':',key )

0 : 12 percentage
1 : 19 province
2 : 25 percentage
3 : 30 percentage
4 : 575 million
5 : additive paper
6 : also said
7 : bad preservation
8 : blamed waste
9 : calling improved
10 : china daily
11 : china fruit
12 : china grain
13 : city showed
14 : consume seven
15 : daily said
16 : daily say
17 : eat 12
18 : fruit output
19 : gave detail
20 : government launched
21 : grain stock
22 : greater production
23 : improved technology
24 : inadequate storage
25 : launched national
26 : left rot
27 : method said
28 : million tonne
29 : national programme
30 : output left
31 : paper blamed
32 : paper gave
33 : percentage china
34 : percentage grain
35 : percentage vegetable
36 : preservation greater
37 : preservation method
38 : production additive
39 : programme reduce
40 : province seven
41 : reduce waste
42 : rot million
43 : said also
44 : said government
45 : said year
46 : say vermin
47 : seven 12
48 : seven city
49 : showed vermin
50 : stock china
51 : stock survey
52 : storage bad
53 

In [20]:
# print ngrams
print(ngrams[0].toarray())

[[2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 2 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


#### Because this is a vocab only from one article, the bag of words for that article will not have any zeros.

In [21]:
# Create a ngram for a sample statement using the same vocabulary
cv.transform(['china governmet launched improved technology']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

bigrams formed are : 'china government', ' government launched', 'launched improved', 'improved technology'. **improved technology** is in index 23, therefore you will see '1' in index 23 above.

In [25]:
# Convert bag of words to a Pandas DataFrame
ngrams_df = pd.DataFrame.sparse.from_spmatrix(ngrams, columns=cv.get_feature_names())

/Users/swapnil/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
ngrams_df

,12 percentage,19 province,25 percentage,30 percentage,575 million,additive paper,also said,bad preservation,blamed waste,calling improved,...,survey 19,technology storage,tonne 25,tonne 30,vegetable paper,vermin consume,vermin eat,waste calling,waste inadequate,year 575
0,2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [27]:
# Add the dataframe to the original dataframe
ngrams_df['category'] = 'grain'


In [28]:
# features and target
ngrams_df

,12 percentage,19 province,25 percentage,30 percentage,575 million,additive paper,also said,bad preservation,blamed waste,calling improved,...,technology storage,tonne 25,tonne 30,vegetable paper,vermin consume,vermin eat,waste calling,waste inadequate,year 575,category
0,2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,grain
